In [171]:
import os
import pandas as pd
import xarray as xr
import urllib
import re
import geopandas as gpd
from shapely.geometry import Point


Food for thought:
    CRW's official gauge for the coral reef watch has boundaries that incr from about
    Long: 131.5 -> 136.5
    Lat: 5 -> 10
    increasing by 2 degrees for 5-8 and by 2 degrees more again for 9-12
    Thus, final map encompasses     
    Long: 127.5 -> 140.5, to 
    Lat: 1->14
    
   
However, the EEZ would be contained in this boundary:
    Long: 129.5088 -> 136.9541
    Lat: 1.6214 -> 11.5587
But, this is a coral product - so EEZ does not equal the area where the corals are

Thus, geoJSON will be made for historic, current, and projection encompassing the area the 9-12 outlook has
since it contains the boundaries for all CRW gauges in addition to the EEZ. 
Which is:
 Long: 127.5 -> 140.5,
 Lat: 1->14


In [172]:
def palau_eez(df):
    df_eez = gpd.read_file('../palauEEZ.geojson')
    eez = df_eez['geometry'].values[0]
    in_palau = []
    longlat = df[['lon','lat']].values.tolist()
    for x in longlat:
        point = Point(x[0],x[1])
        if eez.contains(point) or point.within(eez):
            in_palau.append((x[0],x[1]))
    return df[df[['lon', 'lat']].apply(tuple, axis=1).isin(in_palau)]

In [225]:
def filter_box(df):
    filtered = df[(df["lat"]>= 0.99)&
                                (df["lat"]<=14.01)&(df["lon"]<=140.51)&
                                (df["lon"]>=127.49)]
    return filtered

In [4]:
os.getcwd() 

'/Users/one/Documents/Palau-Graphics/CFS-CRW'

In [ ]:
#https://coralreefwatch.noaa.gov/product/vs/data/palau.txt

In [108]:
#Start folder is 2011, end is 2023
#Start file is 

url = "https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/"

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

rates = ['0'+str(10*x) if x<10 else str(10*x) for x in range(1,11)]

file_years = {}

for x in range(2011,2023+1):
    print(x)
    
    request=urllib.request.Request(url+str(x),None,hdr) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read()
    file_rates = {}
    
    for rate in rates:
        print(rate)
        files = []
        filestarts =[a.start() for a in list(re.finditer("cfsv2_outlook-"+ rate + "perc_4mon-and-wkly_v5_icwk", str(data)))]
        for filestart in filestarts:
            substr = str(data)[filestart:]
            file = substr[:substr.find('.nc')]
            if file + ".nc" not in files:
                files.append(file + ".nc")
        file_rates[rate] = files
    file_years[x] = file_rates

2011
010
020
030
040
050
060
070
080
090
100
2012
010
020
030
040
050
060
070
080
090
100
2013
010
020
030
040
050
060
070
080
090
100
2014
010
020
030
040
050
060
070
080
090
100
2015
010
020
030
040
050
060
070
080
090
100
2016
010
020
030
040
050
060
070
080
090
100
2017
010
020
030
040
050
060
070
080
090
100
2018
010
020
030
040
050
060
070
080
090
100
2019
010
020
030
040
050
060
070
080
090
100
2020
010
020
030
040
050
060
070
080
090
100
2021
010
020
030
040
050
060
070
080
090
100
2022
010
020
030
040
050
060
070
080
090
100
2023
010
020
030
040
050
060
070
080
090
100


In [ ]:
file_years

{2011: {'010': ['cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110703_for_20110717to20111030.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110710_for_20110724to20111030.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110717_for_20110731to20111030.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110724_for_20110807to20111127.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110731_for_20110814to20111127.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110807_for_20110821to20111127.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110814_for_20110828to20111127.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110821_for_20110904to20111225.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110828_for_20110911to20111225.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110904_for_20110918to20111225.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110911_for_20110925to20111225.nc',
   'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110918_for_201

In [112]:
for f in file_years:
    for x in file_years[f]:
        print(file_years[f][x])
        break
    break

['cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110703_for_20110717to20111030.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110710_for_20110724to20111030.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110717_for_20110731to20111030.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110724_for_20110807to20111127.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110731_for_20110814to20111127.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110807_for_20110821to20111127.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110814_for_20110828to20111127.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110821_for_20110904to20111225.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110828_for_20110911to20111225.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110904_for_20110918to20111225.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110911_for_20110925to20111225.nc', 'cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20110918_for_20111002to20120129.nc', 'cfsv2_outlook-010perc_4mon

In [76]:
if not os.path.exists("./CRW-Outlook"):
    os.makedirs("./CRW-Outlook")

In [156]:
for year in file_years:
    if not os.path.exists("./CRW-Outlook/" + "/" + str(year) + "/"):
        os.makedirs("./CRW-Outlook" + "/" + str(year) + "/")
    for file_rate in file_years[year]:
        
        if not os.path.exists("./CRW-Outlook/" + "/" + str(year) + "/" + file_rate):
            os.makedirs("./CRW-Outlook" + "/" + str(year) + "/" + file_rate)
            
        for file in file_years[year][file_rate]:
            if file in os.listdir("./CRW-Outlook/" + str(year) + "/" + file_rate + '/'):
                continue
            filepath = url + str(year) + '/' + file
            print(filepath)
            urllib.request.urlretrieve(filepath,
                               "./CRW-Outlook/" + str(year) + "/" + file_rate + '/' + file)

https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20120212_for_20120226to20120527.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20120219_for_20120304to20120624.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20120226_for_20120311to20120624.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20120304_for_20120318to20120624.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20120311_for_20120325to20120624.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20120318_for_20120401to20120729.nc
https://www.star.nesdi

https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-080perc_4mon-and-wkly_v5_icwk20120129_for_20120212to20120527.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-080perc_4mon-and-wkly_v5_icwk20120205_for_20120219to20120527.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-080perc_4mon-and-wkly_v5_icwk20120212_for_20120226to20120527.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-080perc_4mon-and-wkly_v5_icwk20120219_for_20120304to20120624.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-080perc_4mon-and-wkly_v5_icwk20120226_for_20120311to20120624.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-080perc_4mon-and-wkly_v5_icwk20120304_for_20120318to20120624.nc
https://www.star.nesdi

https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20120115_for_20120129to20120429.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20120122_for_20120205to20120527.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20120129_for_20120212to20120527.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20120205_for_20120219to20120527.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20120212_for_20120226to20120527.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20120219_for_20120304to20120624.nc
https://www.star.nesdi

https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-100perc_4mon-and-wkly_v5_icwk20120101_for_20120115to20120429.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-100perc_4mon-and-wkly_v5_icwk20120108_for_20120122to20120429.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-100perc_4mon-and-wkly_v5_icwk20120115_for_20120129to20120429.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-100perc_4mon-and-wkly_v5_icwk20120122_for_20120205to20120527.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-100perc_4mon-and-wkly_v5_icwk20120129_for_20120212to20120527.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-100perc_4mon-and-wkly_v5_icwk20120205_for_20120219to20120527.nc
https://www.star.nesdi

https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-100perc_4mon-and-wkly_v5_icwk20121223_for_20130106to20130428.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2012/cfsv2_outlook-100perc_4mon-and-wkly_v5_icwk20121230_for_20130113to20130428.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2013/cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20130106_for_20130120to20130428.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2013/cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20130113_for_20130127to20130428.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2013/cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20130120_for_20130203to20130526.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2013/cfsv2_outlook-010perc_4mon-and-wkly_v5_icwk20130127_for_20130210to20130526.nc
https://www.star.nesdi

...

https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-020perc_4mon-and-wkly_v5_icwk20230326_for_20230409to20230730.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-020perc_4mon-and-wkly_v5_icwk20230402_for_20230416to20230730.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-020perc_4mon-and-wkly_v5_icwk20230409_for_20230423to20230730.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-020perc_4mon-and-wkly_v5_icwk20230416_for_20230430to20230730.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-020perc_4mon-and-wkly_v5_icwk20230423_for_20230507to20230827.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-020perc_4mon-and-wkly_v5_icwk20230430_for_20230514to20230827.nc
https://www.star.nesdi

https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-040perc_4mon-and-wkly_v5_icwk20230205_for_20230219to20230528.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-040perc_4mon-and-wkly_v5_icwk20230212_for_20230226to20230528.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-040perc_4mon-and-wkly_v5_icwk20230219_for_20230305to20230625.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-040perc_4mon-and-wkly_v5_icwk20230226_for_20230312to20230625.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-040perc_4mon-and-wkly_v5_icwk20230305_for_20230319to20230625.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-040perc_4mon-and-wkly_v5_icwk20230312_for_20230326to20230625.nc
https://www.star.nesdi

https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-050perc_4mon-and-wkly_v5_icwk20230709_for_20230723to20231029.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-050perc_4mon-and-wkly_v5_icwk20230716_for_20230730to20231029.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-060perc_4mon-and-wkly_v5_icwk20230101_for_20230115to20230430.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-060perc_4mon-and-wkly_v5_icwk20230108_for_20230122to20230430.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-060perc_4mon-and-wkly_v5_icwk20230115_for_20230129to20230430.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-060perc_4mon-and-wkly_v5_icwk20230122_for_20230205to20230528.nc
https://www.star.nesdi

https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20230521_for_20230604to20230924.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20230528_for_20230611to20230924.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20230604_for_20230618to20230924.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20230611_for_20230625to20230924.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20230618_for_20230702to20231029.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-070perc_4mon-and-wkly_v5_icwk20230625_for_20230709to20231029.nc
https://www.star.nesdi

https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20230402_for_20230416to20230730.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20230409_for_20230423to20230730.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20230416_for_20230430to20230730.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20230423_for_20230507to20230827.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20230430_for_20230514to20230827.nc
https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/outlook/v5/nc/v1/outlook/2023/cfsv2_outlook-090perc_4mon-and-wkly_v5_icwk20230507_for_20230521to20230827.nc
https://www.star.nesdi

In [82]:
palau_gauge = "https://coralreefwatch.noaa.gov/product/vs/data/palau.txt"
if not os.path.exists("./CRW-Palau"):
    os.makedirs("./CRW-Palau")
urllib.request.urlretrieve(palau_gauge, "./CRW-Palau/" + "palau.txt")

('./CRW-Palau/palau.txt', <http.client.HTTPMessage at 0x7f92f0be4700>)

In [157]:
folders = os.listdir('./CRW-Outlook/')
folders.sort()
print(folders)

['.DS_Store', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']


In [195]:
time_dict

{}

In [202]:
df_dict

{numpy.datetime64('2011-07-10T12:00:00.000000000'):                    time    lat     lon  CRW_BAA_Week_01  CRW_BAA_Week_02  \
 0   2011-07-10 12:00:00   1.75  132.25              0.0              0.0   
 1   2011-07-10 12:00:00   1.75  132.75              0.0              0.0   
 2   2011-07-10 12:00:00   2.25  130.25              0.0              0.0   
 3   2011-07-10 12:00:00   2.25  130.75              0.0              0.0   
 4   2011-07-10 12:00:00   2.25  131.25              0.0              0.0   
 ..                  ...    ...     ...              ...              ...   
 197 2011-07-10 12:00:00  11.25  133.25              0.0              0.0   
 198 2011-07-10 12:00:00  11.25  133.75              0.0              0.0   
 199 2011-07-10 12:00:00  11.25  134.25              1.0              0.0   
 200 2011-07-10 12:00:00  11.25  134.75              1.0              1.0   
 201 2011-07-10 12:00:00  11.25  135.25              1.0              1.0   
 
      CRW_BAA_Week_03  

In [204]:
time_dict = {}
df_dict = {}
for folder in folders:
    
    time_list = []
    if folder == '.DS_Store':
        continue
    files = os.listdir('./CRW-Outlook/' + folder + '/060/')
    files.sort()
    
    for f in files:
        
        ds = xr.open_dataset('./CRW-Outlook/' + folder + '/060/' + f)
        if ds.time.values[0] in df_dict:
            continue
        #01-20, 4 month composite
        
        #1-4
        df = ds.CRW_BAA_Week_01.to_dataframe()
        df_01 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_02.to_dataframe()
        df_02 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_03.to_dataframe()
        df_03 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_04.to_dataframe()
        df_04 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        mdf = df_01
        mdf['CRW_BAA_Week_02'] = df_02['CRW_BAA_Week_02']
        mdf['CRW_BAA_Week_03'] = df_03['CRW_BAA_Week_03']
        mdf['CRW_BAA_Week_04'] = df_04['CRW_BAA_Week_04']
        mdf['CRW_BAA_Week_01-04'] = mdf[['CRW_BAA_Week_01','CRW_BAA_Week_02','CRW_BAA_Week_03','CRW_BAA_Week_04']].max(axis=1)
        
        
        #5-8
        df = ds.CRW_BAA_Week_05.to_dataframe()
        df_05 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_06.to_dataframe()
        df_06 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_07.to_dataframe()
        df_07 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_08.to_dataframe()
        df_08 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        mdf['CRW_BAA_Week_05'] = df_05['CRW_BAA_Week_05']
        mdf['CRW_BAA_Week_06'] = df_06['CRW_BAA_Week_06']
        mdf['CRW_BAA_Week_07'] = df_07['CRW_BAA_Week_07']
        mdf['CRW_BAA_Week_08'] = df_08['CRW_BAA_Week_08']
        mdf['CRW_BAA_Week_05-08'] = mdf[['CRW_BAA_Week_05','CRW_BAA_Week_06','CRW_BAA_Week_07','CRW_BAA_Week_08']].max(axis=1)
        
        
        #9-12
        df = ds.CRW_BAA_Week_09.to_dataframe()
        df_09 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_10.to_dataframe()
        df_10 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_11.to_dataframe()
        df_11 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_12.to_dataframe()
        df_12 = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        mdf['CRW_BAA_Week_09'] = df_09['CRW_BAA_Week_09']
        mdf['CRW_BAA_Week_10'] = df_10['CRW_BAA_Week_10']
        mdf['CRW_BAA_Week_11'] = df_11['CRW_BAA_Week_11']
        mdf['CRW_BAA_Week_12'] = df_12['CRW_BAA_Week_12']
        mdf['CRW_BAA_Week_09-12'] =  mdf[['CRW_BAA_Week_09','CRW_BAA_Week_10','CRW_BAA_Week_11','CRW_BAA_Week_12']].max(axis=1)
        
        df = ds.CRW_BAA_FourMonth.to_dataframe()
        df_FourMonth = palau_eez(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
    
        mdf['CRW_BAA_FourMonth'] = df_FourMonth['CRW_BAA_FourMonth']    
        
        time_list.append(ds.time.values[0])
        
        df_dict[ds.time.values[0]] = mdf
        print(ds.time.values[0])
        
    time_dict[int(folder)] = time_list

2011-07-10T12:00:00.000000000
2011-07-17T12:00:00.000000000
2011-07-24T12:00:00.000000000
2011-07-31T12:00:00.000000000
2011-08-07T12:00:00.000000000
2011-08-14T12:00:00.000000000
2011-08-21T12:00:00.000000000
2011-08-28T12:00:00.000000000
2011-09-04T12:00:00.000000000
2011-09-11T12:00:00.000000000
2011-09-18T12:00:00.000000000
2011-09-25T12:00:00.000000000
2011-10-02T12:00:00.000000000
2011-10-09T12:00:00.000000000
2011-10-16T12:00:00.000000000
2011-10-23T12:00:00.000000000
2011-10-30T12:00:00.000000000
2011-11-06T12:00:00.000000000
2011-11-13T12:00:00.000000000
2011-11-20T12:00:00.000000000
2011-11-27T12:00:00.000000000
2011-12-04T12:00:00.000000000
2011-12-11T12:00:00.000000000
2011-12-18T12:00:00.000000000
2011-12-25T12:00:00.000000000
2012-01-01T12:00:00.000000000
2012-01-08T12:00:00.000000000
2012-01-15T12:00:00.000000000
2012-01-22T12:00:00.000000000
2012-01-29T12:00:00.000000000
2012-02-05T12:00:00.000000000
2012-02-12T12:00:00.000000000
2012-02-19T12:00:00.000000000
2012-02-26

2016-10-09T12:00:00.000000000
2016-10-16T12:00:00.000000000
2016-10-23T12:00:00.000000000
2016-10-30T12:00:00.000000000
2016-11-06T12:00:00.000000000
2016-11-13T12:00:00.000000000
2016-11-20T12:00:00.000000000
2016-11-27T12:00:00.000000000
2016-12-04T12:00:00.000000000
2016-12-11T12:00:00.000000000
2016-12-18T12:00:00.000000000
2016-12-25T12:00:00.000000000
2017-01-01T12:00:00.000000000
2017-01-08T12:00:00.000000000
2017-01-15T12:00:00.000000000
2017-01-22T12:00:00.000000000
2017-01-29T12:00:00.000000000
2017-02-05T12:00:00.000000000
2017-02-12T12:00:00.000000000
2017-02-19T12:00:00.000000000
2017-02-26T12:00:00.000000000
2017-03-05T12:00:00.000000000
2017-03-12T12:00:00.000000000
2017-03-19T12:00:00.000000000
2017-03-26T12:00:00.000000000
2017-04-02T12:00:00.000000000
2017-04-09T12:00:00.000000000
2017-04-16T12:00:00.000000000
2017-04-23T12:00:00.000000000
2017-04-30T12:00:00.000000000
2017-05-07T12:00:00.000000000
2017-05-14T12:00:00.000000000
2017-05-21T12:00:00.000000000
2017-05-28

2022-01-09T12:00:00.000000000
2022-01-16T12:00:00.000000000
2022-01-23T12:00:00.000000000
2022-01-30T12:00:00.000000000
2022-02-06T12:00:00.000000000
2022-02-13T12:00:00.000000000
2022-02-20T12:00:00.000000000
2022-02-27T12:00:00.000000000
2022-03-06T12:00:00.000000000
2022-03-13T12:00:00.000000000
2022-03-20T12:00:00.000000000
2022-03-27T12:00:00.000000000
2022-04-03T12:00:00.000000000
2022-04-10T12:00:00.000000000
2022-04-17T12:00:00.000000000
2022-04-24T12:00:00.000000000
2022-05-01T12:00:00.000000000
2022-05-08T12:00:00.000000000
2022-05-15T12:00:00.000000000
2022-05-22T12:00:00.000000000
2022-05-29T12:00:00.000000000
2022-06-05T12:00:00.000000000
2022-06-12T12:00:00.000000000
2022-06-19T12:00:00.000000000
2022-06-26T12:00:00.000000000
2022-07-03T12:00:00.000000000
2022-07-10T12:00:00.000000000
2022-07-17T12:00:00.000000000
2022-07-24T12:00:00.000000000
2022-07-31T12:00:00.000000000
2022-08-07T12:00:00.000000000
2022-08-14T12:00:00.000000000
2022-08-21T12:00:00.000000000
2022-08-28

In [213]:
str(time)[:10]

'2011-07-10'

In [214]:
str(time)[:4]

'2011'

In [218]:
if not os.path.exists("./CRW-Outlook/pkl/"):
    os.makedirs("./CRW-Outlook/pkl/")
    
if not os.path.exists("./CRW-Outlook/csv/"):
    os.makedirs("./CRW-Outlook/csv/")

for time in df_dict:
    
    print(time)
    
    if not os.path.exists("./CRW-Outlook/pkl/" + str(time)[:4]):
        os.makedirs("./CRW-Outlook/pkl/" + str(time)[:4])
    
    if not os.path.exists("./CRW-Outlook/csv/" + str(time)[:4]):
        os.makedirs("./CRW-Outlook/csv/" + str(time)[:4])
    
    df_dict[time].to_pickle('./CRW-Outlook/pkl/' + str(time)[:4] + "/CRW_Gauge_"+ str(time)[:10] + '.pkl')
    df_dict[time].to_csv('./CRW-Outlook/csv/' + str(time)[:4] + "/CRW_Gauge_" + str(time)[:10] + '.csv')

2011-07-10T12:00:00.000000000
2011-07-17T12:00:00.000000000
2011-07-24T12:00:00.000000000
2011-07-31T12:00:00.000000000
2011-08-07T12:00:00.000000000
2011-08-14T12:00:00.000000000
2011-08-21T12:00:00.000000000
2011-08-28T12:00:00.000000000
2011-09-04T12:00:00.000000000
2011-09-11T12:00:00.000000000
2011-09-18T12:00:00.000000000
2011-09-25T12:00:00.000000000
2011-10-02T12:00:00.000000000
2011-10-09T12:00:00.000000000
2011-10-16T12:00:00.000000000
2011-10-23T12:00:00.000000000
2011-10-30T12:00:00.000000000
2011-11-06T12:00:00.000000000
2011-11-13T12:00:00.000000000
2011-11-20T12:00:00.000000000
2011-11-27T12:00:00.000000000
2011-12-04T12:00:00.000000000
2011-12-11T12:00:00.000000000
2011-12-18T12:00:00.000000000
2011-12-25T12:00:00.000000000
2012-01-01T12:00:00.000000000
2012-01-08T12:00:00.000000000
2012-01-15T12:00:00.000000000
2012-01-22T12:00:00.000000000
2012-01-29T12:00:00.000000000
2012-02-05T12:00:00.000000000
2012-02-12T12:00:00.000000000
2012-02-19T12:00:00.000000000
2012-02-26

2017-07-09T12:00:00.000000000
2017-07-16T12:00:00.000000000
2017-07-23T12:00:00.000000000
2017-07-30T12:00:00.000000000
2017-08-06T12:00:00.000000000
2017-08-13T12:00:00.000000000
2017-08-20T12:00:00.000000000
2017-08-27T12:00:00.000000000
2017-09-03T12:00:00.000000000
2017-09-10T12:00:00.000000000
2017-09-17T12:00:00.000000000
2017-09-24T12:00:00.000000000
2017-10-01T12:00:00.000000000
2017-10-08T12:00:00.000000000
2017-10-15T12:00:00.000000000
2017-10-22T12:00:00.000000000
2017-10-29T12:00:00.000000000
2017-11-05T12:00:00.000000000
2017-11-12T12:00:00.000000000
2017-11-19T12:00:00.000000000
2017-11-26T12:00:00.000000000
2017-12-03T12:00:00.000000000
2017-12-10T12:00:00.000000000
2017-12-17T12:00:00.000000000
2017-12-24T12:00:00.000000000
2017-12-31T12:00:00.000000000
2018-01-07T12:00:00.000000000
2018-01-14T12:00:00.000000000
2018-01-21T12:00:00.000000000
2018-01-28T12:00:00.000000000
2018-02-04T12:00:00.000000000
2018-02-11T12:00:00.000000000
2018-02-18T12:00:00.000000000
2018-02-25

In [231]:
box_time_dict = {}
box_df_dict = {}
for folder in folders:
    
    time_list = []
    if folder == '.DS_Store':
        continue
    files = os.listdir('./CRW-Outlook/' + folder + '/060/')
    files.sort()
    
    for f in files:
        
        ds = xr.open_dataset('./CRW-Outlook/' + folder + '/060/' + f)
        if ds.time.values[0] in box_df_dict:
            continue
        #01-20, 4 month composite
        
        #1-4
        df = ds.CRW_BAA_Week_01.to_dataframe()
        df_01 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_02.to_dataframe()
        df_02 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_03.to_dataframe()
        df_03 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_04.to_dataframe()
        df_04 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        mdf = df_01
        mdf['CRW_BAA_Week_02'] = df_02['CRW_BAA_Week_02']
        mdf['CRW_BAA_Week_03'] = df_03['CRW_BAA_Week_03']
        mdf['CRW_BAA_Week_04'] = df_04['CRW_BAA_Week_04']
        mdf['CRW_BAA_Week_01-04'] = mdf[['CRW_BAA_Week_01','CRW_BAA_Week_02','CRW_BAA_Week_03','CRW_BAA_Week_04']].max(axis=1)
        
        
        #5-8
        df = ds.CRW_BAA_Week_05.to_dataframe()
        df_05 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_06.to_dataframe()
        df_06 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_07.to_dataframe()
        df_07 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_08.to_dataframe()
        df_08 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        mdf['CRW_BAA_Week_05'] = df_05['CRW_BAA_Week_05']
        mdf['CRW_BAA_Week_06'] = df_06['CRW_BAA_Week_06']
        mdf['CRW_BAA_Week_07'] = df_07['CRW_BAA_Week_07']
        mdf['CRW_BAA_Week_08'] = df_08['CRW_BAA_Week_08']
        mdf['CRW_BAA_Week_05-08'] = mdf[['CRW_BAA_Week_05','CRW_BAA_Week_06','CRW_BAA_Week_07','CRW_BAA_Week_08']].max(axis=1)
        
        
        #9-12
        df = ds.CRW_BAA_Week_09.to_dataframe()
        df_09 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_10.to_dataframe()
        df_10 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_11.to_dataframe()
        df_11 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        df = ds.CRW_BAA_Week_12.to_dataframe()
        df_12 = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
        
        mdf['CRW_BAA_Week_09'] = df_09['CRW_BAA_Week_09']
        mdf['CRW_BAA_Week_10'] = df_10['CRW_BAA_Week_10']
        mdf['CRW_BAA_Week_11'] = df_11['CRW_BAA_Week_11']
        mdf['CRW_BAA_Week_12'] = df_12['CRW_BAA_Week_12']
        mdf['CRW_BAA_Week_09-12'] =  mdf[['CRW_BAA_Week_09','CRW_BAA_Week_10','CRW_BAA_Week_11','CRW_BAA_Week_12']].max(axis=1)
        
        df = ds.CRW_BAA_FourMonth.to_dataframe()
        df_FourMonth = filter_box(df.reset_index()).reset_index(drop=True).sort_values(by=['time','lat','lon']).reset_index(drop=True)
    
        mdf['CRW_BAA_FourMonth'] = df_FourMonth['CRW_BAA_FourMonth']    
        
        time_list.append(ds.time.values[0])
        
        box_df_dict[ds.time.values[0]] = mdf
        print(ds.time.values[0])
        
    box_time_dict[int(folder)] = time_list




2011-07-10T12:00:00.000000000
2011-07-17T12:00:00.000000000
2011-07-24T12:00:00.000000000
2011-07-31T12:00:00.000000000
2011-08-07T12:00:00.000000000
2011-08-14T12:00:00.000000000
2011-08-21T12:00:00.000000000
2011-08-28T12:00:00.000000000
2011-09-04T12:00:00.000000000
2011-09-11T12:00:00.000000000
2011-09-18T12:00:00.000000000
2011-09-25T12:00:00.000000000
2011-10-02T12:00:00.000000000
2011-10-09T12:00:00.000000000
2011-10-16T12:00:00.000000000
2011-10-23T12:00:00.000000000
2011-10-30T12:00:00.000000000
2011-11-06T12:00:00.000000000
2011-11-13T12:00:00.000000000
2011-11-20T12:00:00.000000000
2011-11-27T12:00:00.000000000
2011-12-04T12:00:00.000000000
2011-12-11T12:00:00.000000000
2011-12-18T12:00:00.000000000
2011-12-25T12:00:00.000000000
2012-01-01T12:00:00.000000000
2012-01-08T12:00:00.000000000
2012-01-15T12:00:00.000000000
2012-01-22T12:00:00.000000000
2012-01-29T12:00:00.000000000
2012-02-05T12:00:00.000000000
2012-02-12T12:00:00.000000000
2012-02-19T12:00:00.000000000
2012-02-26

2016-10-23T12:00:00.000000000
2016-10-30T12:00:00.000000000
2016-11-06T12:00:00.000000000
2016-11-13T12:00:00.000000000
2016-11-20T12:00:00.000000000
2016-11-27T12:00:00.000000000
2016-12-04T12:00:00.000000000
2016-12-11T12:00:00.000000000
2016-12-18T12:00:00.000000000
2016-12-25T12:00:00.000000000
2017-01-01T12:00:00.000000000
2017-01-08T12:00:00.000000000
2017-01-15T12:00:00.000000000
2017-01-22T12:00:00.000000000
2017-01-29T12:00:00.000000000
2017-02-05T12:00:00.000000000
2017-02-12T12:00:00.000000000
2017-02-19T12:00:00.000000000
2017-02-26T12:00:00.000000000
2017-03-05T12:00:00.000000000
2017-03-12T12:00:00.000000000
2017-03-19T12:00:00.000000000
2017-03-26T12:00:00.000000000
2017-04-02T12:00:00.000000000
2017-04-09T12:00:00.000000000
2017-04-16T12:00:00.000000000
2017-04-23T12:00:00.000000000
2017-04-30T12:00:00.000000000
2017-05-07T12:00:00.000000000
2017-05-14T12:00:00.000000000
2017-05-21T12:00:00.000000000
2017-05-28T12:00:00.000000000
2017-06-04T12:00:00.000000000
2017-06-11

2022-02-06T12:00:00.000000000
2022-02-13T12:00:00.000000000
2022-02-20T12:00:00.000000000
2022-02-27T12:00:00.000000000
2022-03-06T12:00:00.000000000
2022-03-13T12:00:00.000000000
2022-03-20T12:00:00.000000000
2022-03-27T12:00:00.000000000
2022-04-03T12:00:00.000000000
2022-04-10T12:00:00.000000000
2022-04-17T12:00:00.000000000
2022-04-24T12:00:00.000000000
2022-05-01T12:00:00.000000000
2022-05-08T12:00:00.000000000
2022-05-15T12:00:00.000000000
2022-05-22T12:00:00.000000000
2022-05-29T12:00:00.000000000
2022-06-05T12:00:00.000000000
2022-06-12T12:00:00.000000000
2022-06-19T12:00:00.000000000
2022-06-26T12:00:00.000000000
2022-07-03T12:00:00.000000000
2022-07-10T12:00:00.000000000
2022-07-17T12:00:00.000000000
2022-07-24T12:00:00.000000000
2022-07-31T12:00:00.000000000
2022-08-07T12:00:00.000000000
2022-08-14T12:00:00.000000000
2022-08-21T12:00:00.000000000
2022-08-28T12:00:00.000000000
2022-09-04T12:00:00.000000000
2022-09-11T12:00:00.000000000
2022-09-18T12:00:00.000000000
2022-09-25

FileExistsError: [Errno 17] File exists: './CRW-Outlook/pkl/2011'

In [233]:
if not os.path.exists("./CRW-Outlook/box/"):
    os.makedirs("./CRW-Outlook/box/")

if not os.path.exists("./CRW-Outlook/box/pkl/"):
    os.makedirs("./CRW-Outlook/box/pkl/")
    
if not os.path.exists("./CRW-Outlook/box/csv/"):
    os.makedirs("./CRW-Outlook/box/csv/")

for time in box_df_dict:
    
    print(time)
    
    if not os.path.exists("./CRW-Outlook/box/pkl/" + str(time)[:4]):
        os.makedirs("./CRW-Outlook/box/pkl/" + str(time)[:4])
    
    if not os.path.exists("./CRW-Outlook/box/csv/" + str(time)[:4]):
        os.makedirs("./CRW-Outlook/box/csv/" + str(time)[:4])
    
    box_df_dict[time].to_pickle('./CRW-Outlook/box/pkl/' + str(time)[:4] + "/CRW_Gauge_box_"+ str(time)[:10] + '.pkl')
    box_df_dict[time].to_csv('./CRW-Outlook/box/csv/' + str(time)[:4] + "/CRW_Gauge_box_" + str(time)[:10] + '.csv')

2011-07-10T12:00:00.000000000
2011-07-17T12:00:00.000000000
2011-07-24T12:00:00.000000000
2011-07-31T12:00:00.000000000
2011-08-07T12:00:00.000000000
2011-08-14T12:00:00.000000000
2011-08-21T12:00:00.000000000
2011-08-28T12:00:00.000000000
2011-09-04T12:00:00.000000000
2011-09-11T12:00:00.000000000
2011-09-18T12:00:00.000000000
2011-09-25T12:00:00.000000000
2011-10-02T12:00:00.000000000
2011-10-09T12:00:00.000000000
2011-10-16T12:00:00.000000000
2011-10-23T12:00:00.000000000
2011-10-30T12:00:00.000000000
2011-11-06T12:00:00.000000000
2011-11-13T12:00:00.000000000
2011-11-20T12:00:00.000000000
2011-11-27T12:00:00.000000000
2011-12-04T12:00:00.000000000
2011-12-11T12:00:00.000000000
2011-12-18T12:00:00.000000000
2011-12-25T12:00:00.000000000
2012-01-01T12:00:00.000000000
2012-01-08T12:00:00.000000000
2012-01-15T12:00:00.000000000
2012-01-22T12:00:00.000000000
2012-01-29T12:00:00.000000000
2012-02-05T12:00:00.000000000
2012-02-12T12:00:00.000000000
2012-02-19T12:00:00.000000000
2012-02-26

2017-02-05T12:00:00.000000000
2017-02-12T12:00:00.000000000
2017-02-19T12:00:00.000000000
2017-02-26T12:00:00.000000000
2017-03-05T12:00:00.000000000
2017-03-12T12:00:00.000000000
2017-03-19T12:00:00.000000000
2017-03-26T12:00:00.000000000
2017-04-02T12:00:00.000000000
2017-04-09T12:00:00.000000000
2017-04-16T12:00:00.000000000
2017-04-23T12:00:00.000000000
2017-04-30T12:00:00.000000000
2017-05-07T12:00:00.000000000
2017-05-14T12:00:00.000000000
2017-05-21T12:00:00.000000000
2017-05-28T12:00:00.000000000
2017-06-04T12:00:00.000000000
2017-06-11T12:00:00.000000000
2017-06-18T12:00:00.000000000
2017-06-25T12:00:00.000000000
2017-07-02T12:00:00.000000000
2017-07-09T12:00:00.000000000
2017-07-16T12:00:00.000000000
2017-07-23T12:00:00.000000000
2017-07-30T12:00:00.000000000
2017-08-06T12:00:00.000000000
2017-08-13T12:00:00.000000000
2017-08-20T12:00:00.000000000
2017-08-27T12:00:00.000000000
2017-09-03T12:00:00.000000000
2017-09-10T12:00:00.000000000
2017-09-17T12:00:00.000000000
2017-09-24

2022-08-28T12:00:00.000000000
2022-09-04T12:00:00.000000000
2022-09-11T12:00:00.000000000
2022-09-18T12:00:00.000000000
2022-09-25T12:00:00.000000000
2022-10-02T12:00:00.000000000
2022-10-09T12:00:00.000000000
2022-10-16T12:00:00.000000000
2022-10-23T12:00:00.000000000
2022-10-30T12:00:00.000000000
2022-11-06T12:00:00.000000000
2022-11-13T12:00:00.000000000
2022-11-20T12:00:00.000000000
2022-11-27T12:00:00.000000000
2022-12-04T12:00:00.000000000
2022-12-11T12:00:00.000000000
2022-12-18T12:00:00.000000000
2022-12-25T12:00:00.000000000
2023-01-01T12:00:00.000000000
2023-01-08T12:00:00.000000000
2023-01-15T12:00:00.000000000
2023-01-22T12:00:00.000000000
2023-01-29T12:00:00.000000000
2023-02-05T12:00:00.000000000
2023-02-12T12:00:00.000000000
2023-02-19T12:00:00.000000000
2023-02-26T12:00:00.000000000
2023-03-05T12:00:00.000000000
2023-03-12T12:00:00.000000000
2023-03-19T12:00:00.000000000
2023-03-26T12:00:00.000000000
2023-04-02T12:00:00.000000000
2023-04-09T12:00:00.000000000
2023-04-16